In [1]:
!pip install shap xgboost scikit-learn pandas numpy matplotlib
!pip install opencv-python
! pip install kaggle pandas numpy scikit-learn xgboost shap opencv-python matplotlib



[notice] A new release of pip is available: 24.1.2 -> 25.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.1.2 -> 25.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.1.2 -> 25.0
[notice] To update, run: pip install --upgrade pip


In [2]:
import os
import kaggle

# Set Kaggle credentials (replace 'your-username' with actual Kaggle username)
os.environ['KAGGLE_USERNAME'] = "your-username"
os.environ['KAGGLE_KEY'] = "your-kaggle-api-key"

# Download the dataset
dataset = "rsna-pneumonia-detection-challenge"
os.system(f"kaggle datasets download -d {dataset}")

# Extract the dataset
import zipfile

with zipfile.ZipFile(f"{dataset}.zip", 'r') as zip_ref:
    zip_ref.extractall("pneumonia_data")


403 - Forbidden - Permission 'datasets.get' was denied


FileNotFoundError: [Errno 2] No such file or directory: 'rsna-pneumonia-detection-challenge.zip'

In [12]:
# Set path to the dataset (replace with your actual data path)
BASEPATH = '/kaggle/input/a-pneumonia-detection-challenge/'

def list_subdirectories(base_dir, prefix=''):
    """List all subdirectories in a given base directory."""
    import os
    return [f for f in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, f)) and f.startswith(prefix)]

# Get image folders
image_dirs = BASEPATH + list_subdirectories(BASEPATH)

# Load labels
train_df = pd.read_csv(BASEPATH + 'stage_1_train_labels.csv')
test_df = pd.read_csv(BASEPATH + 'stage_1_test_labels.csv')

# Example: Filter images from the pneumonia and normal categories
desired_classes = ['Pneumonia', 'Normal']
train_df = train_df[train_df['Target'].isin(desired_classes)]
test_df = test_df[test_df['Target'].isin(desired_classes)]

def load_image(image_path, img_size=(128, 128)):
    """Load and preprocess an image."""
    import cv2
    img = cv2.imread(image_path)
    if len(img.shape) == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img, img_size)
    return img

# Load and process training images
train_images = []
for _, row in train_df.iterrows():
    image_path = BASEPATH + row['Path']
    image = load_image(image_path, (64, 64))  # Resize to smaller size for efficiency
    train_images.append(image)

# Convert list of images to numpy array
X_train = np.array(train_images)
y_train = train_df['Target'].values

# Similarly, process test images
test_images = []
for _, row in test_df.iterrows():
    image_path = BASEPATH + row['Path']
    image = load_image(image_path, (64, 64))
    test_images.append(image)

X_test = np.array(test_images)
y_test = test_df['Target'].values

# Normalize pixel values
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

print(f"Training set shape: {X_train.shape}")
print(f"Test set shape: {X_test.shape}")


FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/a-pneumonia-detection-challenge/'

In [11]:
# Import required libraries
import os
import shap
import logging
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

# Configure SHAP logger
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger('shapTutorial')

base_path = os.getcwd()  # Get current working directory
train_df, test_df = load_data(base_path)


FileNotFoundError: Training labels file not found at: /Users/hqin/github/AI4Health/stage_1_train_labels.csv

In [9]:

# Load and Preprocess Data
def load_data(base_path):
    """Load image data from Kaggle's pneumonia detection challenge."""
    # Load labels
    train_labels_csv = os.path.join(base_path, 'stage_1_train_labels.csv')
    if not os.path.exists(train_labels_csv):
        raise FileNotFoundError(f"Training labels file not found at: {train_labels_csv}")
    
    test_labels_csv = os.path.join(base_path, 'stage_1_test_labels.csv')
    if not os.path.exists(test_labels_csv):
        raise FileNotFoundError(f"Testing labels file not found at: {test_labels_csv}")

    train_df = pd.read_csv(train_labels_csv)
    test_df = pd.read_csv(test_labels_csv)

    # Filter images for pneumonia and normal classes (0: Normal, 1: Pneumonia)
    train_df = train_df[train_df['Target'].isin([0, 1])]
    test_df = test_df[test_df['Target'].isin([0, 1])]

    return train_df, test_df

base_path = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
train_df, test_df = load_data(base_path)

def load_image(image_path, img_size=(128, 128)):
    """Load and preprocess an image."""
    img = cv2.imread(image_path)
    if img is None:
        raise FileNotFoundError(f"Image not found: {image_path}")
    
    if len(img.shape) == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img, img_size)
    return img

# Load and process training images
train_images = []
for _, row in train_df.iterrows():
    image_path = os.path.join(base_path, row['Path'])
    image = load_image(image_path, (64, 64))  # Resize to smaller size for efficiency
    train_images.append(image)

X_train = np.array(train_images)
y_train = train_df['Target'].values.astype(int)

# Similarly, process test images
test_images = []
for _, row in test_df.iterrows():
    image_path = os.path.join(base_path, row['Path'])
    image = load_image(image_path, (64, 64))
    test_images.append(image)

X_test = np.array(test_images)
y_test = test_df['Target'].values.astype(int)

# Normalize pixel values
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

print(f"Training set shape: {X_train.shape}")
print(f"Test set shape: {X_test.shape}")

# Split dataset into train and validation set
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42, stratify=y_train
)

# Initialize the model
model = XGBClassifier(
    n_estimators=100,
    max_depth=3,
    learning_rate=0.1,
    gamma=0,
    subsample=1,
    colsample_bytree=1,
    random_state=42
)

# Train the model
logger.info('Starting training...')
model.fit(X_train, y_train)
val_score = model.score(X_val, y_val)
logger.info(f"Validation score: {val_score:.3f}")

# Initialize SHAP explainer with Tree explainer
explainer = shap.TreeExplainer(model)

# Calculate SHAP values for each data point
shap_values = explainer.shap_values(X_test)

# Create summary plot:
shap.summary_plot(shap_values, X_test, show=False)
plt.savefig('shap_summary.png')
plt.close()

# Create dependence plot for a single prediction:
example_idx = 0
sample_shap_values = shap_values[:, example_idx]  # Fixed indexing

shap.dependence_plot(
    'pixel_mean', 
    sample_shap_values,
    X_test, 
    show=False
)
plt.savefig('dependence_plot.png')
plt.close()


NameError: name '__file__' is not defined